In [1]:
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])
train_data = datasets.CIFAR10(root='data',train=True, download=True, transform=transform)
test_data = datasets.CIFAR10(root='data', train=False, download=True, transform=transform)
batch_size = 64
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3,16,3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(16,32,3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        self.fc = nn.Linear(32*8*8, 10)
        
    def forward(self, x):
        x = self.net(x)
        x = x.view(-1, 32*8*8) # x.view(x.size(0),-1)
        return self.fc(x)

In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
model = Model().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_f = nn.CrossEntropyLoss()

epochs = 10
for epoch in range(epochs):
    print(f"Epoch {epoch}")
    for batch_idx, (X, Yd) in enumerate(train_loader):
        optimizer.zero_grad()
        X, Yd = X.to(device), Yd.to(device)
        Y = model(X)
        loss = loss_f(Y, Yd)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            Y = torch.argmax(Y, dim=1)
            correct = (Yd - Y == 0).sum()
            acc = correct / Yd.size(0)
            print(f"Batch Index {batch_idx}, Train Loss: {loss.item():.2e}, Train Accuracy: {acc:.2f}")
    with torch.no_grad():
        loss, correct, tot = 0, 0, 0
        for X, Yd in test_loader:
            X, Yd = X.to(device), Yd.to(device)
            Y = model(X)
            # print(X.shape, Yd.shape, Y.shape)
            loss += loss_f(Y, Yd)
            Y = torch.argmax(Y, dim=1)
            correct += (Yd - Y == 0).sum()
            tot += Yd.size(0)
        loss = loss.item() / len(test_loader)
        acc = correct / tot
        print(f"Test Loss: {loss:.2e}, Test Accuracy: {acc:.2f}")
    break

torch.save(model.state_dict(), 'data/best_mode.pth')
            

cuda
Epoch 0


Batch Index 0, Train Loss: 2.31e+00, Train Accuracy: 0.05
Batch Index 100, Train Loss: 1.51e+00, Train Accuracy: 0.39
Batch Index 200, Train Loss: 1.59e+00, Train Accuracy: 0.47
Batch Index 300, Train Loss: 1.32e+00, Train Accuracy: 0.53
Batch Index 400, Train Loss: 1.35e+00, Train Accuracy: 0.48
Batch Index 500, Train Loss: 1.32e+00, Train Accuracy: 0.48
Batch Index 600, Train Loss: 1.31e+00, Train Accuracy: 0.58
Batch Index 700, Train Loss: 1.24e+00, Train Accuracy: 0.55
Test Loss: 1.25e+00, Test Accuracy: 0.55
